In [5]:
import openai
from dotenv import load_dotenv
import os
from pprint import pprint
import pandas as pd

load_dotenv()
api_key = os.environ['GPT_API_KEY']

openai.api_key = api_key

In [ ]:
file_path = './Historical_Data/Historical Bar Data.csv'

with open(file_path, 'r') as file:
    file_content = file.read()

prompt = f'I want you to analyse the ticker data I have provided below for the ticker "SPY" and analyse, from there I want you to tell me the what time of day to buy and sell and what day of the week in the following format: (Day_of_week[Mon,Tues,etc], Time_of_day_buy[HH:MM], Time_of_day_sell[HH:MM], Risk_Reward[Integer], Stop_loss[percentage as decimal e.g. 0.10 = 10%], Take_Profit[same as stop_loss]). Ticker Data: \n\n{file_content}'


completion = openai.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
        {'role':'system','content': 'You are a Day Trader'},
        {'role':'user','content': prompt}
        ]
)

pprint(completion)

In [15]:
# file_path = './testfile.csv'
file_path = './Historical_Data/Historical Bar Data.csv'

system_prompt = {'role':'system','content': 'You are a Day Trader'}
initial_prompt = {'role':'user','content':'I want you to analyse the ticker data I have provided below for the ticker "SPY" and analyse, from there I want you to tell me the what time of day to buy and sell and what day of the week in the following format: (Day_of_week[Mon,Tues,etc], Time_of_day_buy[HH:MM], Time_of_day_sell[HH:MM], Risk_Reward[Integer], Stop_loss[percentage as decimal e.g. 0.10 = 10%], Take_Profit[same as stop_loss]). I will provide you the data in chunks in subsequent messages'}

message_to_send = []

chunk_size = 1000
 
for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    # print('new chunk')
    chunk_string = chunk.to_csv(index=False)
    message_to_send.append({'role':'user','content': chunk_string})

message_to_send.insert(0,system_prompt)
message_to_send.insert(1,initial_prompt)
# pprint(message_to_send)
completion = openai.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=message_to_send
)

pprint(completion)

new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk


RateLimitError: Error code: 429 - {'error': {'message': 'Request too large for gpt-3.5-turbo in organization org-d6VHS8LxpsDlOucwzRCk4gqg on tokens per min (TPM): Limit 200000, Requested 1111495. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

In [ ]:
assistant = openai.beta.assistants.create(
  name="Financial Analyst Assistant",
  instructions="You are an expert financial analyst. Use you knowledge base to answer questions about audited financial statements.",
  model="gpt-3.5-turbo",
  tools=[{"type": "file_search"},{"type": "code_interpreter"}],
)

vector_store = openai.beta.vector_stores.create(name="Financial Statements")

# file_to_upload = "./Historical_Data/Historical Bar Data.csv"
# file_to_upload = "./Historical Bar Data.txt"
file_to_upload = "./testfile3.txt"
file_streams = open(file_to_upload, 'rb')

file_batch = openai.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)

# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

assistant = openai.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

# Upload the user provided file to OpenAI
message_file = openai.files.create(
  file=open(file_to_upload, "rb"), purpose="assistants", content_type="text/plain"
)
 
# Create a thread and attach the file to the message
thread = openai.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "How many shares of SPY were outstanding at the end of of October 2023?",
      # Attach the new file to the message.
      "attachments": [
        { "file_id": message_file.id, "tools": [{"type": "file_search"}] }
      ],
    }
  ]
)
 
# The thread now has a vector store with that file in its tool resources.
print(thread.tool_resources.file_search)
# print(thread)

run = openai.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

messages = list(openai.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

message_content = messages[0].content[0].text
annotations = message_content.annotations
citations = []
for index, annotation in enumerate(annotations):
    message_content.value = message_content.value.replace(annotation.text, f"[{index}]")
    if file_citation := getattr(annotation, "file_citation", None):
        cited_file = openai.files.retrieve(file_citation.file_id)
        citations.append(f"[{index}] {cited_file.filename}")

print(message_content.value)
print("\n".join(citations))

In [ ]:
import os

# Paths to test files
csv_file_path = "./testfile.csv"
text_file_path = "./testtextfile.txt"

# Check if files exist and print their contents
for file_path in [csv_file_path, text_file_path]:
    if os.path.exists(file_path):
        print(f"File {file_path} exists. Contents:")
        with open(file_path, 'r') as file:
            print(file.read())
    else:
        print(f"File {file_path} does not exist.")

In [ ]:
# Upload CSV file
try:
    with open(csv_file_path, "rb") as file:
        message_file_csv = openai.File.create(
            file=file,
            purpose="assistants"
        )
        print("CSV file uploaded successfully. File ID:", message_file_csv.id)
except Exception as e:
    print("Error during CSV file upload:", e)

# Upload Text file
try:
    with open(text_file_path, "rb") as file:
        message_file_txt = openai.File.create(
            file=file,
            purpose="assistants"
        )
        print("Text file uploaded successfully. File ID:", message_file_txt.id)
except Exception as e:
    print("Error during text file upload:", e)